# Logistic Regression

In [1]:
import torch
import torch.nn as nn
import torchvision

# Some transformations of images
import torchvision.transforms as transforms

In [2]:
# Hyper-parameters
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root="../data",
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)

test_dataset = torchvision.datasets.MNIST(root="../data",
                                         train=False,
                                         transform=transforms.ToTensor())

0it [00:00, ?it/s]

9920512it [00:02, 3372870.25it/s]                             


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:02, 15652.67it/s]                         
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 895670.15it/s]                              
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:01, 6707.04it/s]             

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [4]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

In [5]:
# Logistic Regression model
model = nn.Linear(input_size, num_classes)

In [6]:
# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
# Train the model
total_step = len(train_loader)
total_step

600

In [21]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28 * 28)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}"
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.1780
Epoch [1/5], Step [200/600], Loss: 2.1221
Epoch [1/5], Step [300/600], Loss: 2.0158
Epoch [1/5], Step [400/600], Loss: 1.9437
Epoch [1/5], Step [500/600], Loss: 1.8678
Epoch [1/5], Step [600/600], Loss: 1.7974
Epoch [2/5], Step [100/600], Loss: 1.7012
Epoch [2/5], Step [200/600], Loss: 1.5993
Epoch [2/5], Step [300/600], Loss: 1.6590
Epoch [2/5], Step [400/600], Loss: 1.5859
Epoch [2/5], Step [500/600], Loss: 1.5611
Epoch [2/5], Step [600/600], Loss: 1.4550
Epoch [3/5], Step [100/600], Loss: 1.3571
Epoch [3/5], Step [200/600], Loss: 1.3934
Epoch [3/5], Step [300/600], Loss: 1.3037
Epoch [3/5], Step [400/600], Loss: 1.3875
Epoch [3/5], Step [500/600], Loss: 1.3055
Epoch [3/5], Step [600/600], Loss: 1.1994
Epoch [4/5], Step [100/600], Loss: 1.2455
Epoch [4/5], Step [200/600], Loss: 1.2515
Epoch [4/5], Step [300/600], Loss: 1.0961
Epoch [4/5], Step [400/600], Loss: 1.0954
Epoch [4/5], Step [500/600], Loss: 1.1103
Epoch [4/5], Step [600/600], Loss:

In [26]:
# Test the model
# In test phase, we don't need to compute gradients (for memory. efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    
    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 83 %


In [28]:
# Save the model checkpoint
torch.save(model.state_dict(), 'MNIST_model.ckpt')

##### NOTE 1 array.reshape(-1, M)

In [18]:
import numpy as np

a = np.arange(6)
a.shape

(6,)

In [19]:
a = a.reshape(-1, 6)
a.shape

(1, 6)

tensor([[ 5.3884e-01, -1.3481e+00, -1.0058e+00,  3.0111e+00, -1.8852e+00,
          1.2214e+00, -1.6251e+00,  7.6262e-01,  9.1857e-01, -4.0217e-01],
        [-6.2323e-01,  6.7765e-01,  4.4092e-01, -3.6608e-01, -6.1942e-01,
          2.2997e-02, -3.1251e-01, -8.4028e-01,  1.6883e+00, -2.5687e-01],
        [ 3.6454e+00, -1.8560e+00, -1.0469e-01, -1.3125e-01, -8.6578e-01,
          1.5658e+00, -7.9346e-01, -1.2703e+00,  5.9517e-01, -7.7735e-01],
        [-3.4119e-01, -1.9348e+00, -4.6738e-01, -1.7801e+00,  1.3659e+00,
         -5.8388e-01,  1.5332e-01,  3.6009e-01,  4.0970e-01,  1.8045e+00],
        [-1.4308e+00,  1.1214e-01,  6.8054e-01,  2.1675e-01, -7.6274e-01,
         -2.5912e-01, -5.8612e-01,  2.7065e-01,  7.3579e-01,  2.0035e-01],
        [-1.1267e+00, -1.2300e+00, -5.2176e-01, -8.7693e-01,  1.3518e+00,
         -7.1581e-03, -1.2976e-01,  5.5302e-01,  4.9525e-01,  1.1622e+00],
        [-1.4480e+00, -1.2768e-01, -5.4856e-01, -2.5905e-01,  7.5688e-01,
          2.2068e-01, -1.5097e-0